School of Computer Sciences, USM<br>Semester 2, 2020/2021

# CDS513: Predictive Business Analytics - Group Project

## Data Preprocessing for Market Basket Analysis (MBA)

##### Project Title
\> Improving Sales Performance of the Ecommerce Website for an Electronics Store using Predictive Business Analytics Techniques 

##### Group No
\> Group 3 \[Lee Yong Meng (P-COM0012/20) | Lee Kar Choon (P-COM0130/19) | Lim Hang Thing (P-COM0143/20)\]

##### Dataset
\> Purchase data: [Ecommerce Purchase History from Electronics Store](https://www.kaggle.com/mkechinov/ecommerce-purchase-history-from-electronics-store)

<img alt="Credit card" src="https://images.unsplash.com/photo-1563013544-824ae1b704d3?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1950&q=80">

Photo by <a href="https://unsplash.com/@rupixen?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">rupixen.com</a> on <a href="https://unsplash.com/s/photos/credit-card?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>


## Overview

**Part 1: [Load Data](#load)**
- 1.1. [Purchase Data](#load-purchase)
- 1.2. [Explore Data](#load-explore)

**Part 2: [Transform Data](#transform)**
- 2.1. [Create New Column: `product_name`](#transform-new-col)
- 2.2. [Drop rows with `price == NaN`](#transform-drop-price)
- 2.3. [Filter Purchase Data](#transform-filter)
- 2.4. [Generate Pivot Table for MBA ](#transform-pivot)
- 2.5. [Save Data](#transform-save)

In [1]:
# Import Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Keep track of the processing time
from datetime import datetime

print_start = lambda: print(f"Start time: {datetime.now()}")
print_end = lambda: print(f"End time: {datetime.now()}")

***

# 1. Load Data <a name="load"></a>

## 1.1. Purchase Data <a name="load-purchase"></a>

Load the purchase data.

In [2]:
# ============================================================
# Load purchase data
# ============================================================

df_purchase = pd.read_csv("src/kz.csv")

display(df_purchase.head())
display(df_purchase.shape)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


(2633521, 8)

## 1.2. Explore Data <a name="load-explore"></a>

In [3]:
df_purchase.describe(include='all')

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
count,2633521,2.633521e+06,2.633521e+06,2.201567e+06,2021319,2127516,2.201567e+06,5.641690e+05
unique,1316174,NaN,NaN,NaN,510,23021,NaN,NaN
top,1970-01-01 00:33:40 UTC,NaN,NaN,NaN,electronics.smartphone,samsung,NaN,NaN
freq,19631,NaN,NaN,NaN,357682,358928,NaN,NaN
mean,NaN,2.361783e+18,1.674080e+18,2.273827e+18,NaN,NaN,1.540932e+02,1.515916e+18
std,NaN,1.716538e+16,3.102249e+17,2.353247e+16,NaN,NaN,2.419421e+02,2.379057e+07
min,NaN,2.294360e+18,1.515966e+18,2.268105e+18,NaN,NaN,0.000000e+00,1.515916e+18
25%,NaN,2.348807e+18,1.515966e+18,2.268105e+18,NaN,NaN,1.456000e+01,1.515916e+18
50%,NaN,2.353254e+18,1.515966e+18,2.268105e+18,NaN,NaN,5.553000e+01,1.515916e+18
75%,NaN,2.383131e+18,1.515966e+18,2.268105e+18,NaN,NaN,1.967400e+02,1.515916e+18


Check the information of each column in `df_purchase`.

In [4]:
df_purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2633521 entries, 0 to 2633520
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   order_id       int64  
 2   product_id     int64  
 3   category_id    float64
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        float64
dtypes: float64(3), int64(2), object(3)
memory usage: 160.7+ MB


Check number of missing values in `df_purchase`.

In [5]:
np.sum(df_purchase.isna())

event_time             0
order_id               0
product_id             0
category_id       431954
category_code     612202
brand             506005
price             431954
user_id          2069352
dtype: int64

**Note:** Notice how `category_id` and `price_id` have the same number of missing values (`431,954`). It is therefore safe to assume that there is some systematic error when collecting data for these products. In our case, we drop these rows because they don't really help in our analysis.

Display unique values of each attribute.

In [6]:
display(df_purchase[['product_id', 'category_id', 'category_code', 'brand', 'user_id']].nunique())
display(df_purchase['category_code'].unique()[:10])

product_id       25113
category_id        900
category_code      510
brand            23021
user_id          98262
dtype: int64

array(['electronics.tablet', 'electronics.audio.headphone', nan,
       'furniture.kitchen.table', 'electronics.smartphone',
       'appliances.kitchen.refrigerators', 'appliances.personal.scales',
       'electronics.video.tv', 'computers.components.cpu',
       'computers.notebook'], dtype=object)

Perform some analysis on the purchase data.

##### Q: What is the total price of the products in a selected order?

In [7]:
df_purchase[df_purchase['order_id'] == 2353219851624907454]['price'].sum()

1216.7

##### Q: How many unique orders are here? 

In [8]:
# How many unique orders are here?
df_purchase['order_id'].nunique()

1435266

##### Q: What is the total gross sales generated by the electronic store from January to November 2020?

In [9]:
# How much is the gross sales of the electronic store?
df_purchase['price'].sum()

339246427.70999986

***

# 2. Transform Data <a name="transform"></a>

## 2.1. Create New Column: `product_name` <a name="transform-new-col"></a>

Impute missing data - assign values to missing `category_code` and `brand`:

- `category_code = 'unknown_category'`
- `brand = 'unknown_brand'`

Then, combine `category_code` and `brand` to form new value for each record - `product_name`.

In [10]:
%%time

print_start()

# ============================================================
# Derive new columns
# ============================================================

df_purchase['category_code'] = df_purchase['category_code'].fillna('unknown_category')
df_purchase['brand'] = df_purchase['brand'].fillna('unknown_brand')

display(np.sum(df_purchase[['category_code', 'brand']].isna()))

df_purchase['product_name'] = df_purchase['category_code'] + '-' + df_purchase['brand']
display(df_purchase.head())

print_end()

Start time: 2021-06-27 17:51:30.527552


category_code    0
brand            0
dtype: int64

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,product_name
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18,electronics.tablet-samsung
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18,electronics.tablet-samsung
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18,electronics.audio.headphone-huawei
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18,electronics.audio.headphone-huawei
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,unknown_category,karcher,217.57,1.515916e+18,unknown_category-karcher


End time: 2021-06-27 17:51:31.490014
Wall time: 962 ms


Check number of unique product names.

In [12]:
%%time

print_start()
display(df_purchase[['order_id', 'product_name']].nunique())
print_end()

Start time: 2021-06-27 17:51:55.401834


order_id        1435266
product_name      37631
dtype: int64

End time: 2021-06-27 17:51:55.986834
Wall time: 585 ms


## 2.2. Drop Rows with `price == NaN` <a name="transform-drop-price"></a>

Assume that the rows without `price` is not as valuable for our analysis. We need `price` to filter values, and also for time series analysis (later).

In [13]:
# Drop rows with price == NaN
df_purchase_with_price = df_purchase[~df_purchase['price'].isna()]

display(df_purchase_with_price.head())
display(df_purchase_with_price.shape)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,product_name
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18,electronics.tablet-samsung
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18,electronics.tablet-samsung
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18,electronics.audio.headphone-huawei
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18,electronics.audio.headphone-huawei
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,unknown_category,karcher,217.57,1.515916e+18,unknown_category-karcher


(2201567, 9)

## 2.3. Filter Purchase Data <a name="transform-filter"></a>

Group records by `order_id` - it helps to identify how many products (non-unique) are purchased within each order.

In [14]:
df_purchase_by_order = (df_purchase[['order_id', 'event_time']]
                        .groupby('order_id')
                        .count()
                        .sort_values(by='event_time', ascending=False))

display(df_purchase_by_order)

,event_time
order_id,
2388440981134393883,61
2388440981134689974,53
2348791291912913068,48
2353219851624907454,39
2348807739909603922,36
...,...
2353283278804353192,1
2353283278837907625,1
2353283278946959530,1


At this point, there are too many items to be analysed. Generating dataset for Market Basket Analysis (MBA) yields `ValueError` as such:

    ValueError: Unstacked DataFrame is too big, causing int32 overflow

Perform analysis on the purchase data. 

##### Q: What is the number of orders with at least 2 products (non-unique)?

In [15]:
df_purchase_by_order[df_purchase_by_order.event_time > 1].count()

event_time    563515
dtype: int64

### 2.3.1. Generate Total Sales by Products

Generate total sales by `product_name` (i.e., combinations of `category_code` and `brand`)

##### Q: What are the total sales by products and the units of each product sold from January to November 2020 ?

In [17]:
df_purchase['unit_sold'] = 1

total_sales_by_product = (df_purchase
                          .groupby('product_name')
                          .sum()
                          .sort_values(by='price', ascending=False)[['price', 'unit_sold']])

display(total_sales_by_product)

,price,unit_sold
product_name,,
electronics.smartphone-samsung,4.498434e+07,178903.0
electronics.smartphone-apple,3.535255e+07,46030.0
electronics.video.tv-samsung,1.267467e+07,22698.0
electronics.video.tv-lg,1.049990e+07,19234.0
computers.notebook-lenovo,1.025737e+07,17886.0
...,...,...
18.29-1515915625498716803,0.000000e+00,1.0
18.29-1515915625500117609,0.000000e+00,1.0
18.29-1515915625504462182,0.000000e+00,1.0


### 2.3.2. Generate Total Sales by Orders

Generate total sales by `order_id`.

##### Q: What are the total sales for each order/transaction from January to October 2020 ?

In [18]:
total_sales_by_order = (df_purchase
                        .groupby('order_id')
                        .sum()
                        .sort_values(by='price', ascending=False)[['price', 'unit_sold']])

total_sales_by_order

,price,unit_sold
order_id,,
2353234364520727464,52141.15,3.0
2339244674033647628,18699.03,2.0
2339940319191106553,13310.16,1.0
2353235498996073224,12698.98,7.0
2348787642851656266,12404.97,7.0
...,...,...
2348779258186301610,0.00,1.0
2348805576227554277,0.00,1.0
2388440981134550764,0.00,1.0


### 2.3.3. Filter Records

#### `MIN_PRODUCT_SALES`: Minimum amount of total sales by each product

Generate set of products (`product_name`) with total sales generated from January to November 2020 not less than `MIN_PRODUCT_SALES`.

In [19]:
# ------------------------------------------------------------
# Generate set of product (total sales >= MIN_PRODUCT_SALES)
# ------------------------------------------------------------

# - Minimum gross sales amount per item in the purchase data
MIN_PRODUCT_SALES = 30_000

product_sales = total_sales_by_product[total_sales_by_product['price'] >= MIN_PRODUCT_SALES]
display(product_sales)

set_product = set(product_sales.index)
print(f"There are {len(list(set_product))} products sold with at least {MIN_PRODUCT_SALES} of sales amount.")

,price,unit_sold
product_name,,
electronics.smartphone-samsung,4.498434e+07,178903.0
electronics.smartphone-apple,3.535255e+07,46030.0
electronics.video.tv-samsung,1.267467e+07,22698.0
electronics.video.tv-lg,1.049990e+07,19234.0
computers.notebook-lenovo,1.025737e+07,17886.0
...,...,...
electronics.smartphone-unknown_brand,3.027897e+04,5128.0
construction.tools.screw-ava,3.018357e+04,5309.0
appliances.kitchen.refrigerators-electrolux,3.007536e+04,25.0


There are 480 products sold with at least 30000 of sales amount.


#### `MIN_ORDER_PRICE`: Minimum amount of total sales by each order/transaction

Generate set of orders (`order_id`) from January to November 2020 with sales amount not less than `MIN_ORDER_PRICE`.

In [20]:
# ------------------------------------------------------------
# Generate set of order ID (amount >= MIN_ORDER_PRICE)
# ------------------------------------------------------------

# Minimum gross sales amount per item in the purchase data
MIN_ORDER_PRICE = 200

order_price = total_sales_by_order[total_sales_by_order['price'] >= MIN_ORDER_PRICE]
display(order_price)

set_order = set(order_price.index)

print(f"There are {len(list(set_order))} transactions with amount not less than {MIN_ORDER_PRICE}.")

,price,unit_sold
order_id,,
2353234364520727464,52141.15,3.0
2339244674033647628,18699.03,2.0
2339940319191106553,13310.16,1.0
2353235498996073224,12698.98,7.0
2348787642851656266,12404.97,7.0
...,...,...
2388440981134608386,200.05,2.0
2353286435714368404,200.05,2.0
2324587826118656338,200.05,2.0


There are 502383 transactions with amount not less than 200.


#### Filtering process

In [21]:
# ------------------------------------------------------------
# Define conditions for filtered records
# ------------------------------------------------------------

df_purchase_mba = df_purchase[['order_id', 'product_name']]

# Define condition to filter records
df_purchase_mba['popular_product'] = df_purchase_mba['product_name'].apply(lambda x: x in set_product)
df_purchase_mba['order_over_min_sales'] = df_purchase_mba['order_id'].apply(lambda x: x in set_order)

CONDITION = ((df_purchase_mba['popular_product'] == True) & 
             (df_purchase_mba['order_over_min_sales'] == True))

# ------------------------------------------------------------
# Group filtered records by `order_id` and `product_name` 
# ------------------------------------------------------------

df_purchase_group_by_order_n_product = (df_purchase_mba[CONDITION]
                                        .groupby(['order_id', 'product_name'])
                                        .count())

print(f"# records: {df_purchase_group_by_order_n_product.shape[0]}")

df_purchase_group_by_order_n_product.head(20)

<ipython-input-21-e62aad05c88b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_purchase_mba['popular_product'] = df_purchase_mba['product_name'].apply(lambda x: x in set_product)
<ipython-input-21-e62aad05c88b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_purchase_mba['order_over_min_sales'] = df_purchase_mba['order_id'].apply(lambda x: x in set_order)


# records: 859810


,,popular_product,order_over_min_sales
order_id,product_name,,
2294359932054536986,electronics.tablet-samsung,2,2
2294584263154074236,unknown_category-karcher,1,1
2295740594749702229,electronics.smartphone-apple,4,4
2295902490203259134,appliances.kitchen.refrigerators-lg,1,1
2296400480990920715,electronics.video.tv-samsung,1,1
2296628237930857206,computers.components.cpu-intel,3,3
2297016008231092565,computers.notebook-asus,1,1
2297174044555871159,computers.peripherals.monitor-samsung,2,2
2297252054407578606,computers.peripherals.printer-epson,3,3


##### Q: How many unique transactions/orders and products are in the list?

In [22]:
num_order = len(list(set([order_id 
                          for order_id, _ 
                          in df_purchase_group_by_order_n_product.index.values])))

num_product = len(list(set([product_id 
                            for _, product_id
                            in df_purchase_group_by_order_n_product.index.values])))

print(f"# unique orders: {num_order}\n# unique products: {num_product}")

# unique orders: 499886
# unique products: 480


## 2.4. Generate Pivot Table for MBA <a name="transform-pivot"></a>

Apply `pivot_table` to transform records into transaction-item utility matrix for MBA.

In [24]:
%%time

print_start()

# ============================================================
# Generate pivot table for MBA
# ============================================================

# Convert grouped records into pivot table
df_purchase_mba_matrix = (df_purchase_group_by_order_n_product
                          .pivot_table(index='order_id', 
                                       columns='product_name', 
                                       values='popular_product'))

# Display numerical pivot table (with missing values)
print("Pivot table (numerical):")
display(df_purchase_mba_matrix.head(5))

# Convert cell values into Boolean:
# ... False: missing value; True: otherwise.
df_purchase_mba_matrix = ~df_purchase_mba_matrix.isna()

# Display binomial pivot table
print("Pivot table (binomial):")
display(df_purchase_mba_matrix.head(5))

print_end()

Start time: 2021-06-27 17:57:02.624705
Pivot table (numerical):


product_name,accessories.bag-samsung,accessories.bag-transcend,apparel.glove-nintendo,apparel.glove-sony,apparel.glove-xbox,apparel.shirt-ggg,apparel.tshirt-ggg,appliances.environment.air_conditioner-ava,appliances.environment.air_conditioner-beko,appliances.environment.air_conditioner-lg,...,unknown_category-tailg,unknown_category-technodom,unknown_category-tefal,unknown_category-thefaceshop,unknown_category-transcend,unknown_category-unknown_brand,unknown_category-vitek,unknown_category-x-game,unknown_category-xiaomi,unknown_category-zhiyun
order_id,,,,,,,,,,,,,,,,,,,,,
2294359932054536986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2294584263154074236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2295740594749702229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2295902490203259134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2296400480990920715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Pivot table (binomial):


product_name,accessories.bag-samsung,accessories.bag-transcend,apparel.glove-nintendo,apparel.glove-sony,apparel.glove-xbox,apparel.shirt-ggg,apparel.tshirt-ggg,appliances.environment.air_conditioner-ava,appliances.environment.air_conditioner-beko,appliances.environment.air_conditioner-lg,...,unknown_category-tailg,unknown_category-technodom,unknown_category-tefal,unknown_category-thefaceshop,unknown_category-transcend,unknown_category-unknown_brand,unknown_category-vitek,unknown_category-x-game,unknown_category-xiaomi,unknown_category-zhiyun
order_id,,,,,,,,,,,,,,,,,,,,,
2294359932054536986,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2294584263154074236,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2295740594749702229,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2295902490203259134,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2296400480990920715,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


End time: 2021-06-27 17:57:07.605954
Wall time: 4.98 s


## 2.5. Save Data <a name="transform-save"></a>

# *The size of the output file is 1.45GB.*

In [25]:
%%time

# ============================================================
# Save data
# ============================================================

print_start()
df_purchase_mba_matrix.to_csv('output/mba_matrix.csv')
print_end()

Start time: 2021-06-27 17:58:02.420068
End time: 2021-06-27 17:58:44.313932
Wall time: 41.9 s
